In [1]:
import numpy as np
from numpy import sqrt, absolute, log, ones, zeros, array, transpose
from numpy.linalg import multi_dot
from numpy.random import sample
from scipy.optimize import curve_fit
from scipy.stats import chi2
from numdifftools.nd_algopy import Gradient, Derivative
from inspect import signature

from scipy import stats
from jax import random
from jax import numpy as jnp
from warnings import warn

from jax.config import config
config.update("jax_enable_x64", True)

import numpy as np
import pylab as pl
from jax import grad
from jax import numpy as jnp
from jax import random
from scipy import stats\

import line_profiler
%load_ext line_profiler

from analisi_errori import *

In [11]:
x = np.array([1,2,5,5])
dx = x*0.01

propagate_errors(f, x, dx)

array([0.01990827, 0.07930196, 0.4997989 , 0.50276367])

In [3]:
def f(x):
    return x**2

cov = np.random.normal(0, 1, (3, 3))
cov = cov@cov.T
x = [1]*3

dy(f, x, 1e-6*cov)

array([[ 1.59921062e-05, -1.29630637e-05,  1.22674165e-06],
       [-1.29630637e-05,  1.66506017e-05, -7.01510621e-06],
       [ 1.22674165e-06, -7.01510621e-06,  6.00985934e-06]])

In [50]:
%lprun -f compute_error compute_error(f,x,cov)

Timer unit: 1e-06 s

Total time: 1.82342 s
File: <ipython-input-49-1c9a3e85c56e>
Function: compute_error at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def compute_error(f, x, pcov, n_samples=10000000):
     8         1       1442.0   1442.0      0.1      x, pcov = jnp.array(x, dtype=jnp.float64), jnp.array(pcov, dtype=jnp.float64)
     9                                           
    10         1          4.0      4.0      0.0      seed = int(time.time())
    11         1       2850.0   2850.0      0.2      key = random.PRNGKey(seed)
    12                                               
    13         1        115.0    115.0      0.0      samples = random.multivariate_normal(key, mean=x, cov=pcov, shape=(int(n_samples),))
    14         1    1803234.0 1803234.0     98.9      y = f(samples.T)
    15         1      11236.0  11236.0      0.6      mean = f(x)
    16                                           
    1

In [ ]:
# Creates a multivariate uniform distrubution, mainly for fun
def multivariate_uniform(mean, cov, n_samples):
    mean = np.array(mean)
    xs = np.empty((n_samples, cov.shape[0]))
    chol = np.linalg.cholesky(cov)
    for i in range(cov.shape[0]):
        xs[:, i] = np.random.uniform(-np.sqrt(12)/2, np.sqrt(12)/2, size=n_samples)
    
    return xs@chol.T + mean

cov = np.random.normal(0, 1, (2, 2))
cov = cov@cov.T
mean = [0]*2
x, y = multivariate_uniform(mean, cov, 1000).T
xn, yn = np.random.multivariate_normal(mean, cov, 1000).T

pl.plot(x, y, '.')
pl.plot(xn, yn, '.')

In [25]:
# Faster random number generation using jax

key = random.PRNGKey(0)
mean = jnp.array([0]*3)
cov = jnp.array(cov)

%time x=np.random.multivariate_normal(mean, cov, size=1000000)
%time x=random.multivariate_normal(key, mean=mean, cov=cov, shape=(int(1e8),))

CPU times: user 134 ms, sys: 111 µs, total: 134 ms
Wall time: 108 ms
CPU times: user 38.4 ms, sys: 11.6 ms, total: 50 ms
Wall time: 14.4 ms
